# HAMLD

Generate a surface code noisy line and sample it.

In [1]:
import numpy as np
import stim
circuit = stim.Circuit.generated("surface_code:rotated_memory_x", 
                                 distance=3, 
                                 rounds=1, 
                                 after_clifford_depolarization=0.05)
num_shots = 1000
model = circuit.detector_error_model(decompose_errors=False, flatten_loops=True)
sampler = circuit.compile_detector_sampler()
syndrome, actual_observables = sampler.sample(shots=num_shots, separate_observables=True)


Construct an MLD decoder and perform decoding.

In [2]:
import hamld
mld_decoder = hamld.HAMLD(detector_error_model=model, order_method='mld', slice_method='no_slice')
predicted_observables = mld_decoder.decode_batch(syndrome)
num_mistakes = np.sum(np.any(predicted_observables != actual_observables, axis=1))

print(f"{num_mistakes}/{num_shots}")

69/1000


Construct an HMLD decoder and perform decoding.

In [3]:
import hamld
hmld_decoder = hamld.HAMLD(detector_error_model=model, order_method='greedy', slice_method='no_slice')
predicted_observables = hmld_decoder.decode_batch(syndrome)
num_mistakes = np.sum(np.any(predicted_observables != actual_observables, axis=1))

print(f"{num_mistakes}/{num_shots}")

69/1000


Construct an HAMLD decoder and perform decoding.

In [4]:
import hamld

hamld_decoder =  hamld.HAMLD(detector_error_model=model,
                            order_method='greedy',
                            slice_method='no_slice',
                            use_approx = True,
                            approximatestrategy = "hyperedge_topk",
                            approximate_param = 100,
                            contraction_code = "normal",
                            accuracy = "float64",
                            priority = -2,
                            priority_topk= 150)

predicted_observables = hamld_decoder.decode_batch(syndrome)
num_mistakes = np.sum(np.any(predicted_observables != actual_observables, axis=1))
                            
print(f"{num_mistakes}/{num_shots}")

69/1000


Data preprocessing: group together the same (syndrome, logical observable) and decode once, implementing a decoding method similar to a look-up table.

After data preprocessing, decoding is performed.

It does not affect the decoding accuracy, but can reduce the time required for QEC experiments.

For more information, please refer to the code in the benchmarking directory.

In [5]:
sample = np.hstack((syndrome, actual_observables))
len_syndrome = syndrome.shape[1]
unique_sample, counts = np.unique(sample, axis = 0, return_counts=True)
num_unique_sample = unique_sample.shape[0]
unique_syndrome = unique_sample[:,:len_syndrome]
unique_actual_observables = unique_sample[:,len_syndrome:]

hmld_decoder = hamld.HAMLD(detector_error_model=model, order_method='greedy', slice_method='no_slice')

unique_predicted_observables = hmld_decoder.decode_batch(unique_syndrome)
mistakes_mask = np.any(unique_predicted_observables != unique_actual_observables, axis=1)
num_mistakes = np.sum(mistakes_mask * counts)

print(f"{num_mistakes}/{num_shots}")

69/1000


Decode a syndrome, where after each decoding, the output is a probability distribution. We need to select the logical operation corresponding to the syndrome with a large probability for error correction.

In [6]:
syndrome = syndrome[0]
syndrome

array([False, False, False,  True, False, False, False, False])

In [7]:
mld_decoder.decode(syndrome)

(array([False]),
 {(False,
   False,
   False,
   True,
   False,
   False,
   False,
   False,
   True): 0.0010887631938922256,
  (False,
   False,
   False,
   True,
   False,
   False,
   False,
   False,
   False): 0.019538367135331285},
 0.947216933402039)

In [8]:
hmld_decoder.decode(syndrome)

(array([False]),
 {(False,
   False,
   False,
   True,
   False,
   False,
   False,
   False,
   True): 0.0010887631938922256,
  (False,
   False,
   False,
   True,
   False,
   False,
   False,
   False,
   False): 0.019538367135331285},
 0.947216933402039)

In [9]:
hamld_decoder.decode(syndrome)

(array([False]),
 {(False,
   False,
   False,
   True,
   False,
   False,
   False,
   False,
   True): 0.0010887631938922256,
  (False,
   False,
   False,
   True,
   False,
   False,
   False,
   False,
   False): 0.019538367135331285},
 0.947216933402039)

The decoding tasks between different syndromes are entirely independent, so in specific experimental tests, we can use Python's multiprocessing to perform parallel decoding.  

The speedup ratio depends on the number of CPU cores. The `parallel_decode_batch` function can also take a `num_workers` parameter; otherwise, it will use half of `multiprocessing.cpu_count()`.  

Through parallel decoding, the time required for error correction experiments can be significantly reduced in large-scale quantum error correction simulations.

In [10]:
circuit = stim.Circuit.generated("surface_code:rotated_memory_x", 
                                distance=3, 
                                rounds=1, 
                                after_clifford_depolarization=0.05)
num_shots = 10000
model = circuit.detector_error_model(decompose_errors=False, flatten_loops=True)
sampler = circuit.compile_detector_sampler()
syndrome, actual_observables = sampler.sample(shots=num_shots, separate_observables=True)

import hamld
hmld_decoder = hamld.HAMLD(detector_error_model=model, order_method='greedy', slice_method='no_slice')
predicted_observables = hmld_decoder.parallel_decode_batch(syndrome)

num_mistakes = np.sum(np.any(predicted_observables != actual_observables, axis=1))

print(f"{num_mistakes}/{num_shots}")

628/10000


In certain scenarios, we can enable detailed analysis by outputting the probability distribution. By setting output_prob=True, the system will not only provide the predicted logical error for syndrome correction, but also return the corresponding probability distribution for further investigation.

In [11]:
circuit = stim.Circuit.generated("surface_code:rotated_memory_x", 
                                distance=3, 
                                rounds=1, 
                                after_clifford_depolarization=0.05)
num_shots = 10
model = circuit.detector_error_model(decompose_errors=False, flatten_loops=True)
sampler = circuit.compile_detector_sampler()
syndrome, actual_observables = sampler.sample(shots=num_shots, separate_observables=True)

import hamld
hmld_decoder = hamld.HAMLD(detector_error_model=model, order_method='greedy', slice_method='no_slice')
predicted_observables, prob_dists = hmld_decoder.decode_batch(syndrome, output_prob=True)

num_mistakes = np.sum(np.any(predicted_observables != actual_observables, axis=1))

print(f"{num_mistakes}/{num_shots}")
print(f"predicted_observables: {predicted_observables.shape}")
print(f"prob_dists: {prob_dists.shape}")

1/10
predicted_observables: (10, 1)
prob_dists: (10, 2)


The output_prob parameter remains configurable in parallel execution.

In [12]:
circuit = stim.Circuit.generated("surface_code:rotated_memory_x", 
                                distance=3, 
                                rounds=1, 
                                after_clifford_depolarization=0.05)
num_shots = 100
model = circuit.detector_error_model(decompose_errors=False, flatten_loops=True)
sampler = circuit.compile_detector_sampler()
syndrome, actual_observables = sampler.sample(shots=num_shots, separate_observables=True)

import hamld
hamld_decoder = hamld.HAMLD(detector_error_model=model, order_method='greedy', slice_method='no_slice')
predicted_observables, prob_dists = hamld_decoder.parallel_decode_batch(syndrome, output_prob=True)

num_mistakes = np.sum(np.any(predicted_observables != actual_observables, axis=1))

print(f"{num_mistakes}/{num_shots}")
print(f"predicted_observables: {predicted_observables.shape}")
print(f"prob_dists: {prob_dists.shape}")

4/100
predicted_observables: (100, 1)
prob_dists: (100, 2)


## 调用高性能实现的C++接口

参考README.md编译c++接口

In [25]:
# from hamld import HAMLDCpp_from_file
# from hamld.benchmark import generate_detector_error_model_path

In [ ]:
# d = 3
# r = 1
# # real p is 10/10000
# p = 10
# noise_model = "si1000"
# error_type = "Z"
# related_path = "../data/external/epmld_experiment_data/epmld_paper_experiment/overall_performance/surface_code"
# have_stabilizer = False

# approximatestrategy = "hyperedge_topk"
# approximate_param = 3
# priority = 0
# priority_topk = 3
# dem_file = generate_detector_error_model_path(d = d, r=r, p=p, noise_model=noise_model,
#                                               error_type=error_type, decomposed_error = False,
#                                               related_path=related_path, have_stabilizer = have_stabilizer)
# decoder =  HAMLDCpp_from_file(dem_path=str(dem_file), 
#                               approx_strategy=str(approximatestrategy), 
#                               approx_param=float(approximate_param), 
#                               priority=int(priority), 
#                               priority_topk = int(priority_topk),
#                               use_heuristic = False, alpha = float(0.05))

# # if have_stabilizer = False, surface code.
# detector_number = int((d**2-1)*r - (d**2-1)/2)
# random_syndrome = np.random.randint(0, 2, detector_number)
# print("random_syndrome:", random_syndrome)
# predicted_observable, prob_dist, relative_correct_probability = decoder.decode(random_syndrome)
# print("predicted_observable:", predicted_observable)
# # The syndrome is remapped to a new one, so sometimes random_syndrome and the syndrome in prob_dist differ.
# print("prob_dist:", prob_dist)

random_syndrome: [0 0 1 0]
predicted_observable: [False]
prob_dist: {'0010': 0.009524717155758559}
